<a href="https://colab.research.google.com/github/pbhatt48/langchain-demos/blob/main/page_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.1/271.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
import subprocess
subprocess.run(["huggingface-cli", "login", "--token", "<Your Token>"])

In [3]:
## Import all the libraries
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')


In [4]:
model="meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

In [5]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=10000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [6]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [7]:
##Adding memory
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
memory = ConversationBufferMemory()
prompt_template_name = PromptTemplate(input_variables =['product'],
                                      template="WHat would be the good company names for {product}")
chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
chain.run('underwear')

' and sleepwear?\n\nAnswer: Here are some suggestions for good company names for underwear and sleepwear:\n\nUnderwear:\n\n1. BarelyThere: This name plays on the idea of being comfortable and confident in your underwear.\n2. Pantastic: A playful name that combines the words "pants" and "fantastic."\n3. UndieLove: A cute and catchy name that conveys a sense of affection for your underwear.\n4. BoxedUp: This name could work for a company that sells underwear in a variety of styles and sizes.\n5. Skivvies: A playful name that sounds fun and carefree.\n6. ComfyTights: A straightforward name that conveys the idea of comfortable underwear.\n7. HuggiePants: A name that evokes the feeling of being snug and cozy in your underwear.\n8. B bottoms: A fun and playful name that sounds like a brand you could trust.\n9. Pantheon: A name that conjures up images of a high-end, stylish brand.\n10. Drawers: A simple and straightforward name that works for a brand that sells underwear in a variety of style

In [8]:
## Document loaders
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/wb-axp1.pdf")
pages = loader.load()
pages

[Document(page_content='1964: AMERICAN EXPRESS\u2003/three.taboldstyle/five.taboldstyle\nFigure 4.1.was committing fraud. Allied went bankrupt, and the warehouse, when collecting the collateral, realized that the tanks that it had thought were /f_illed with valuable salad oil were instead /f_illed with seawater. With an estimated liability of up to /dollar.oldstyle/one.oldstyle/five.oldstyle/zero.oldstyle million, Allied and the American Express subsidiary /f_iled for bankruptcy protection. Whether American Express, the parent company, was liable was uncertain. Nevertheless, fearing damage to American Express’s reputation, CEO and President Howard Clark issued a statement that American Express felt morally bound to see that such liabili-ties were satis/f_ied./one.oldstyle/T_he word on the street was that American Express could face insol-vency. Its stock, which had been trading at /dollar.oldstyle/six.oldstyle/zero.oldstyle per share before the news, dropped to /dollar.oldstyle/three.o

In [9]:
pages[0]

Document(page_content='1964: AMERICAN EXPRESS\u2003/three.taboldstyle/five.taboldstyle\nFigure 4.1.was committing fraud. Allied went bankrupt, and the warehouse, when collecting the collateral, realized that the tanks that it had thought were /f_illed with valuable salad oil were instead /f_illed with seawater. With an estimated liability of up to /dollar.oldstyle/one.oldstyle/five.oldstyle/zero.oldstyle million, Allied and the American Express subsidiary /f_iled for bankruptcy protection. Whether American Express, the parent company, was liable was uncertain. Nevertheless, fearing damage to American Express’s reputation, CEO and President Howard Clark issued a statement that American Express felt morally bound to see that such liabili-ties were satis/f_ied./one.oldstyle/T_he word on the street was that American Express could face insol-vency. Its stock, which had been trading at /dollar.oldstyle/six.oldstyle/zero.oldstyle per share before the news, dropped to /dollar.oldstyle/three.ol

In [10]:
##Adding memory
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
memory = ConversationBufferMemory()
prompt_template_name = PromptTemplate(input_variables =['content'],
                                      template="Can you summarize and polish the language for the text: {content}")
chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)


In [11]:
chain.run(pages[0].page_content)

"\nPolishing the language of the text can help make it clearer and more concise. Here is an example of how the text could be rewritten with more refined language:\n1964: AMERICAN EXPRESS FACES FINANCIAL SCANDAL\nIn 1964, American Express faced a major financial scandal that threatened its very existence. An investigation revealed that one of its subsidiaries, Allied Business Credit, was involved in a fraudulent scheme and had incurred significant liabilities. Allied had been operating a scheme where it would buy expensive items, such as tanks, fill them with seawater, and then sell them to unsuspecting buyers at inflated prices. When the scheme was discovered, American Express and its subsidiary filed for bankruptcy protection, leaving millions of dollars in liabilities unpaid. The news sent shockwaves through the financial industry, and American Express's stock price plummeted, losing nearly half of its value in a matter of days.\nAs the scandal unfolded, fears grew that American Expr

In [12]:
#chain.run(pages.page_content)

In [13]:
# import textwrap
# wrapped_text = textwrap.fill(sum, width=100)
# print(wrapped_text +'\n\n')

In [14]:
# for i in range(len(pages)):
#   chain.run(pages[i].page_content)

In [15]:
chain.run(pages[0].page_content)

"\nThe language in the provided text is a bit complex and convoluted, so I've simplified and polished it for easier understanding:\n1964: American Express was embroiled in a scandal when it was discovered that one of its subsidiaries, Allied, had been committing fraud. Allied went bankrupt, and the warehouse realized that the tanks it had thought were filled with valuable salad oil were actually filled with seawater. American Express faced an estimated liability of up to $5 million. The company and its CEO, Howard Clark, issued a statement expressing a moral obligation to see that the liabilities were satisfied. The stock price dropped amid bad publicity, and shareholders later sued Clark for oﬀering $6 million to settle claims. Buﬀett, sensing opportunity beneath the scandal, began his primary research by speaking to customers and vendors in Omaha, including restaurants and restaurant customers, and he found that their using habits had not changed despite the scandal. He concluded tha